In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode
from syft.core.frameworks.torch.tensor import _MPCTensor
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

bob.add_workers([alice, james])
alice.add_workers([bob, james])
james.add_workers([bob, alice])

In [2]:
x = torch.FloatTensor([-1,2,3,4]).send(bob)

In [3]:
bob._objects

{74465717639: [_LocalTensor - id:74465717639 owner:bob]}

In [4]:
xfp = x.fix_precision()

fixing actual precision
[Head of chain]
[syft.core.frameworks.torch.tensor.FloatTensor with no dimension]

<class 'syft.core.frameworks.torch.tensor._FixedPrecisionTensor'>
<class 'syft.core.frameworks.torch.tensor.LongTensor'>
returning mpc


In [5]:
smpc = xfp.share(alice, james)

In [6]:
type(bob._objects[smpc.id_at_location])

syft.core.frameworks.torch.tensor._FixedPrecisionTensor

In [7]:
bob._objects[smpc.id_at_location].ser(False)

{'___FixedPrecisionTensor__': {'base': 10,
  'child': {'__LongTensor__': {'child': {'___MPCTensor__': {'id': 3542891464,
      'owner': 0,
      'shares': {'__LongTensor__': {'child': {'___GeneralizedPointerTensor__': {'id': 1446307943,
          'owner': 0,
          'pointer_tensor_dict': {'alice': {'___PointerTensor__': {'id': 7584561645,
             'id_at_location': 4423006959,
             'location': 'alice',
             'owner': 0,
             'torch_type': 'syft.LongTensor'}},
           'james': {'___PointerTensor__': {'id': 8056838812,
             'id_at_location': 9111753040,
             'location': 'james',
             'owner': 0,
             'torch_type': 'syft.LongTensor'}}},
          'torch_type': 'syft.LongTensor'}},
        'data': [],
        'torch_type': 'syft.LongTensor',
        'type': 'syft.core.frameworks.torch.tensor.LongTensor'}},
      'torch_type': 'syft.LongTensor'}},
    'data': [],
    'torch_type': 'syft.LongTensor',
    'type': 'syft.core.fram

In [8]:
y = x + x

In [5]:
y = x + x

In [6]:
y.decode()

AttributeError: '_PointerTensor' object has no attribute 'decode'

In [ ]:
shared_x.

In [4]:
# x.fix_precision()

In [3]:
y = x + x

In [4]:
y.get().decode()


-2
 4
 6
 8
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [12]:
x.send(bob)

FloatTensor[_PointerTensor - id:9534671634 owner:0 loc:bob id@loc:81106429269]

In [10]:
x.get()


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [5]:
# x = torch.FloatTensor([1,2,3,4])

# x.send(bob)
# x_shared = x.share(alice, james)

In [3]:
x2 = x_shared.get()

In [4]:
y = x2 + x2

In [5]:
y.get()


 2
 4
 6
 8
[syft.core.frameworks.torch.tensor.LongTensor of size 4]

In [6]:
x.child

[_PointerTensor - id:4314925884 owner:0 loc:bob id@loc:7183012622]

In [7]:
self = x.child

cmd = {}
cmd['command'] = "share"
cmd['args'] = [alice.id, james.id]
cmd['kwargs'] = {}
cmd['has_self'] = True
cmd['self'] = self

shared_x = self.handle_call(cmd, self.owner).wrap(True)

returning mpc


In [5]:
out = shared_x.get()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [4]:
y = out + out

In [5]:
y.get()


 2
 4
 6
 8
[syft.core.frameworks.torch.tensor.LongTensor of size 4]

In [6]:
mpcser = mpc.ser(True)
mpcser

newmpc = decoder.python_decode(mpcser)

In [7]:
y = newmpc + newmpc

In [8]:
y.get()


 2
 4
 6
 8
[syft.core.frameworks.torch.tensor.LongTensor of size 4]

In [2]:
dct = {'alice': {'___PointerTensor__': {'owner': 0, 'id': 7576607770, 'location': 'alice', 'id_at_location': 23898948540, 'torch_type': 'syft.LongTensor'}}}['alice']

In [3]:
dct

{'___PointerTensor__': {'id': 7576607770,
  'id_at_location': 23898948540,
  'location': 'alice',
  'owner': 0,
  'torch_type': 'syft.LongTensor'}}

In [4]:
sy._PointerTensor.deser(dct['___PointerTensor__'], worker=me, acquire=False)

[_PointerTensor - id:9099805064 owner:0 loc:0 id@loc:7576607770]

In [5]:
y = torch.FloatTensor([1,2,3,4]).ser(False)
decoder = encode.PythonJSONDecoder(worker=me, acquire=True)
out = decoder.python_decode(y)
out




 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

LongTensor[_PointerTensor - id:1266418860 owner:0 loc:bob id@loc:10452873429]

In [43]:
shared_x

[_PointerTensor - id:3905345674 owner:0 loc:bob id@loc:4315372835]

RecursionError: maximum recursion depth exceeded

In [11]:
bob._objects

{4155439184: [_MPCTensor - id:4155439184 owner:bob],
 83404070888: [_LocalTensor - id:83404070888 owner:0]}

In [12]:
alice._objects

{75896689532: [_LocalTensor - id:75896689532 owner:alice]}

In [13]:
james._objects

{76269744433: [_LocalTensor - id:76269744433 owner:james]}

In [14]:
shared_x.get()

RecursionError: maximum recursion depth exceeded while calling a Python object